# Dalai in Python

### Start server in terminal

    npx dalai serve

### Install python-socketio

In [1]:
# update installer
!pip install -qU pip
# there's also an async version
!pip install -qU python-socketio[client]

### Connect to server

In [2]:
import socketio
# open a client
sio = socketio.Client()

In [3]:
# try to connect
try:
    sio.connect('http://localhost:3000')
except Exception as e:
    print(e)
    pass

### Listen for anything

Catch all listener for things we don't have other handlers for. This helped me figure out what was happening at first.

In [4]:
# Uncomment to activate: 
# @sio.on('*')
def catch_all(event, data):
    from pprint import pformat
    print('unhandled event:',event)
    print('unhandled data:\n', pformat(data))
    print()


For example, that printed many versions of following, with usually one word in 'response' at a time:

    unhandled event: result
    unhandled data:
     {'request': {'debug': False,
                 'id': 'TS-1679621517384-99808',
                 'model': 'alpaca.13B',
                 'models': ['alpaca.7B', 'llama.7B', 'alpaca.13B'],
                 'n_predict': 25,
                 'prompt': 'Below is an instruction that describes a task, paired '
                           'with an input that provides further context. Write a '
                           'response that appropriately completes the request.\n'
                           '\n'
                           '### Instruction:\n'
                           'Tell me a story!\n'
                           '### Response:\n',
                 'repeat_last_n': 64,
                 'repeat_penalty': 1.3,
                 'seed': -1,
                 'temp': '.8',
                 'threads': 4,
                 'top_k': 40,
                 'top_p': 0.9},
     'response': 'Once'}

### Listen for results better

Because results come in a little weird, with one word at a time, we need to make some other solution. Here's one hacky one:

In [5]:
# Global vars to remember the results by
RESULTS = {}
REQ_IDS = []

# When we get a result message in particular
@sio.on('result')
def on_result(data):
    global RESULTS, REQ_IDS
    
    # Get this request ID
    req_id = data.get('request',{}).get('id')
    new_word = data.get('response','')
    # If there are both
    if req_id and new_word:
        # And if it's not already in results
        if not req_id in RESULTS:
            # then initially stuff it with this data
            RESULTS[req_id] = data
            # and add this request id to the last 
            REQ_IDS.append(req_id)
        # If it's already in results
        else:
            # then simply add the new response word
            RESULTS[req_id]['response'] += new_word    
    
    # Should we also print the word? Why not
    print(new_word, end='', flush=True)

In [6]:
# Quick function to get latest result
def get_latest_result():
    # If there are results
    if REQ_IDS and RESULTS:
        # get latest id
        req_id = REQ_IDS[-1]
        # get result dictionary from latest id as key
        result = RESULTS[req_id]
        # return result
        return result
    # otherwise return empty dictionary
    return {}

### Send requests

So to send requests back, I think we can use that. Feel free to modify:

In [10]:
# A request dictionary
request_d = {'debug': False,
              'id': 'TS-1679621517384-99808',
              'model': 'alpaca.13B',
              'models': ['alpaca.7B', 'llama.7B', 'alpaca.13B'],
              'n_predict': 25,
              'prompt': 'Below is an instruction that describes a task, paired '
                        'with an input that provides further context. Write a '
                        'response that appropriately completes the request.\n\n'
                        '### Instruction:\n'
                        'Tell me a story!\n\n'
                        '### Response:\n',
              'repeat_last_n': 64,
              'repeat_penalty': 1.3,
              'seed': -1,
              'temp': '.8',
              'threads': 4,
              'top_k': 40,
              'top_p': 0.9}

#### Actually send it!

Use the emit function with 'request' as the event name, and the request dictionary as the message.

In [11]:
# Emit the request
sio.emit('request', request_d)

# ...in a few seconds should start printing according to our on_result handler above

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Tell me a story!

### Response:
Once upon a time there was once a girl named Jenna who lived in a small town by the sea. She had dreamed of seeing new places, but her family couldn’t afford to travel far from home so she settled for exploring around where they were living instead. One day while walking along the coastline with an old friend, Jenna noticed that something seemed off and decided to follow it until a bright light came into view!
She was soon taken on adventure through time itself as her eyes began adjusting from one era of life to another. She met interesting characters throughout history who each had their own stories to tell about the world they lived in, but Jenna’s favorite by far were those that took place near waterways and shores!
Finally after a few weeks she came upon an old man sitting on h

In [12]:
# Get latest result as dictionary
get_latest_result()

{'response': "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\r\n\r\n### Instruction:\r\nTell me a story!\r\n### Response:\r\nOnce upon a time there was little girl who lived in a small village on top of a hill. Every day she would walk down to collect water from the well and watch all her friends play games together by the river at sunset, but she couldn't join them because no one had ever taught how swim before! One night as she sat watching everyone else have fun in their own way without her, a mysterious voice called out asking if anyone wanted to learn to dive into uncharted waters and explore what lay beneath. The little girl looked up at the sky and took courage that this was going be something extraordinary...\r\nAnd so began an epic tale of bravery!\n\n<end>Below is an instruction that describes a task, paired with an input that provides further context. Write a respon